In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WRITING_GRPO_CONFIG
from trl_wrapper.wrapper_config import SMOL_LM_135M

cfg = WRITING_GRPO_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_completion_length = 32
cfg.max_prompt_length = 32
cfg.max_samples = 10

/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 05-04 21:03:48 [importing.py:17] Triton not installed or not compatible; certain GPU-related functions will not be available.
WARNING 05-04 21:03:48 [importing.py:28] Triton is not installed. Using dummy decorators. Install it via `pip install triton` to enable kernelcompilation.
INFO 05-04 21:03:48 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-04 21:03:48 [__init__.py:239] Automatically detected platform cpu.


2025-05-04 21:03:49,704	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
wrapper = TrainerWrapper(cfg, True)

2025-05-04 21:03:50.800 | INFO     | trl_wrapper.trainer_wrapper:__init__:303 - Setting padding side to: right


In [3]:
wrapper.init_model()

2025-05-04 21:03:50.849 | INFO     | trl_wrapper.trainer_wrapper:init_model:318 - Using device: mps
2025-05-04 21:03:50.850 | INFO     | trl_wrapper.trainer_wrapper:init_model:332 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [18]:
wrapper.init_data_module()

2025-05-04 21:19:17.449 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:358 - Using chat template override: smollmv2
Map: 100%|██████████| 659/659 [00:00<00:00, 22142.31 examples/s]
2025-05-04 21:19:18.132 | INFO     | synthetic_data.creative_writing_bench.slop_score:load_slop_list_to_set:20 - Loaded 1000 items from ../dataset_files//slop_list.json
2025-05-04 21:19:18.132 | INFO     | synthetic_data.creative_writing_bench.slop_score:load_slop_list_to_set:20 - Loaded 200 items from ../dataset_files//slop_list_bigrams.json
2025-05-04 21:19:18.133 | INFO     | synthetic_data.creative_writing_bench.slop_score:load_slop_list_to_set:20 - Loaded 200 items from ../dataset_files//slop_list_trigrams.json


In [5]:
wrapper.data_module.config.run_mode

'notebook'

In [17]:
wrapper.init_trainer()

2025-05-04 21:19:14.436 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:396 - Saving output to: ./runs/05-04-21-19-390448-smollm2-135m-instruct--style-focus
2025-05-04 21:19:14.437 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:419 - Initializing trainer, run_name: 05-04-21-19-390448-smollm2-135m-instruct--style-focus, wandb project: writing-grpo
2025-05-04 21:19:14.437 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:502 - Using vllm: False


In [16]:
wrapper.data_module.train_dataset[0]

{'completion': '```xml\n<analysis>\n   <a>\n   This paragraph details Gary Gygax\'s early efforts to create and test *Dungeons & Dragons*. It describes the creation of Castle Greyhawk and the initial playtests with his children and friends, highlighting their enthusiasm and the game\'s immediate appeal.\n   </a>\n   <b>\n   The writing style is informative and narrative, presenting historical details with a sense of storytelling. Examples include: "The first people to play it were Gygax’s eleven-year-old son, Ernie, and nine-year-old daughter, Elise." and "Fatherly duties completed, Gygax returned to his office and designed another level of the dungeons."\n   </b>\n   <c>\n   The tone is enthusiastic and slightly nostalgic, conveying the excitement surrounding the birth of *D&D*. The sentence "“The reaction . . . was instant enthusiasm,” wrote Gygax" directly indicates this enthusiasm.\n   </c>\n   <d>\n   The main content focuses on the creation and initial playtesting of *D&D*. Key t

In [25]:
wrapper.train()

2025-05-04 21:26:16.680 | INFO     | trl_wrapper.trainer_wrapper:train:650 - Starting training.
2025-05-04 21:26:18.831 | INFO     | dataset.writing:_run_in_thread:99 - No loop in thread, creating a new one.
2025-05-04 21:26:18.831 | INFO     | dataset.writing:_run_in_thread:103 - Running coroutine in background thread 17909755904 using loop <_UnixSelectorEventLoop running=False closed=False debug=False>
2025-05-04 21:26:18.832 | INFO     | synthetic_data.tasks.writing:score_writing:416 - Judging 32 completions with gemini-2.0-flash
2025-05-04 21:26:20.804 | INFO     | dataset.writing:_run_in_thread:116 - Background thread 17909755904 finished execution.
2025-05-04 21:26:20.805 | INFO     | dataset.writing:llm_judge_func:181 - Sample scores - totals: [0.24, 0.24, 0.43, 0.39, 0.24, 0.24, 0.33, 0.43, 0.43, 0.18, 0.24, 0.24, 0.24, 0.24, 0.32, 0.24, 0.24, 0.24, 0.24, 0.24, 0.04, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.04, 0.04, 0.24, 0.24]
2025-05-04 21:26:20.805 | INFO     | dataset.w

Step,Training Loss
1,0.000000


2025-05-04 21:26:26.604 | INFO     | dataset.writing:_run_in_thread:99 - No loop in thread, creating a new one.
2025-05-04 21:26:26.604 | INFO     | dataset.writing:_run_in_thread:103 - Running coroutine in background thread 17672728576 using loop <_UnixSelectorEventLoop running=False closed=False debug=False>
2025-05-04 21:26:26.605 | INFO     | synthetic_data.tasks.writing:score_writing:416 - Judging 32 completions with gemini-2.0-flash
2025-05-04 21:26:28.325 | INFO     | dataset.writing:_run_in_thread:116 - Background thread 17672728576 finished execution.
2025-05-04 21:26:28.325 | INFO     | dataset.writing:llm_judge_func:181 - Sample scores - totals: [0.24, 0.23, 0.04, 0.04, 0.24, 0.48, 0.24, 0.17, 0.24, 0.24, 0.24, 0.24, 0.24, 0.43, 0.43, 0.25, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.43, 0.24, 0.24, 0.24, 0.43, 0.24, 0.43, 0.24]
2025-05-04 21:26:28.326 | INFO     | dataset.writing:llm_judge_func:183 - 	Adherence to Instructions: [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0,

KeyboardInterrupt: 

In [ ]:
import pandas as pd
dataset_pd = pd.read_parquet(
    "../dataset_files/gutenberg_backtranslate_from_txt.parquet"
)
completion, instruction = dataset_pd.loc[500]['text'], dataset_pd.loc[500]['instruction']

In [ ]:
max_score = 0
for i in range(10000):
    completion = dataset_pd.loc[i]['text']
    score = wrapper.data_module.bench.calculate_slop_index(completion)
    if score > max_score:
        max_score = score
        print(score)
print(max_score)

In [ ]:
wrapper.data_module.reward_functions()[0]([instruction], [completion])